In [16]:
from __future__ import print_function
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import versor as vsr
import hyperdual
from versor.drawing import *

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Inf']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [30]:
th = pi/6
B = vsr.EGA(vsr.Biv(1,0,0)) * np.tan(th/2)
B
one = vsr.EGA(1,0,0,0,0,0,0,0)

In [33]:
vsr.EGA(vsr.Vec(1,2,3)).spin((one + B) * (one - B).inv())

EGA: [ 0 2.2 0.13 3 0 0 0 0 ]

In [35]:
vsr.EGA(vsr.Vec(1,2,3)).spin(vsr.EGA(vsr.Biv(-th,0,0).exp()))

EGA: [ 0 2.2 0.13 3 0 0 0 0 ]

In [2]:
rotor = (vsr.Biv(1,0,0).unit() * (pi/6)).exp()

In [3]:
B = vsr.EGA(rotor.log())
B

EGA: [ 0 0 0 0 -0.52 0 0 0 ]

In [4]:
one = vsr.EGA(1,0,0,0,0,0,0,0)

In [5]:
R2 = (one + B) * (one - B).inv()
R2 * R2.rev()

EGA: [ 1 0 0 0 0 0 0 0 ]

In [6]:
M = vsr.Vec(1,2,3).trs() * rotor
M

Mot: [ 0.87 -0.5 0 0 -0.93 -0.62 -1.3 0.75 ]

In [7]:
Bm = M.log()
Bm

Dll: [ -0.5236 0 0 -0.977 -0.6451 -1.5 ]

In [14]:
MextE = (Bm ^ Bm) * 0.5 + Bm + 1

In [15]:
MextE * MextE.rev()

Mot: [ 1.3 0 0 0 -5.6e-17 0 5.6e-17 0 ]

In [11]:
(Bm ^ Bm) ^ Bm

Mot: [ 0 0 0 0 0 0 0 0 ]

In [ ]:
Bm = vsr.Mot(0,Bm[0],Bm[1],Bm[2],Bm[3],Bm[4],Bm[5],0)

In [ ]:
def Cayley(B):
    one = onem = vsr.Mot(1,0,0,0,0,0,0,0)
    B = vsr.Mot(0,Bm[0],Bm[1],Bm[2],Bm[3],Bm[4],Bm[5],0)
    B2 = B * B
    B20 = vsr.Mot(B2[0],0,0,0,0,0,0,0)
    B22 = vsr.Mot(0,B2[1],B2[2],B2[3],B2[4],B2[5],B2[6],0)
    B24 = vsr.Mot(0,0,0,0,0,0,0,B2[7])
    print((one - B20 + B24))
    return (one + B + B + B2) * (one - B20 + B24) * (1.0 / (1 - B2[0])**2)

In [ ]:
Mc = Cayley(M.log())
np.array(Mc)

In [ ]:
(onem - Bm).inv()

In [ ]:
(onem - Bm.rev()) * (onem - Bm * Bm).inv()

In [ ]:
np.t

In [ ]:
Mc.log().biv().duale().unit()

In [ ]:
M.log().biv().duale().unit()

In [ ]:
F = (Bm * Bm + Bm * Bm ) * 0.5
F[0] *= -1
F[0] += 1
F

In [ ]:
M2 = (onem + Bm) * (onem + Bm) * F * (1.0 / F[0]**2)
vsr.Vec(0,0,0).null().spin(M2)

In [ ]:
vsr.Vec(0,0,0).null().spin(M)

In [ ]:
Bd = (M2.rev() * M).log()

In [ ]:
M.log()

In [ ]:
Bd

In [ ]:
onem = vsr.Mot(1,0,0,0,0,0,0,0)

In [ ]:
R2 = (onem - Bm).inv() * (onem + Bm)
R2 * R2.rev()

In [ ]:
rotor

In [ ]:
n_points=10
sigma = 0.01
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).unit() for i in range(n_points)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]
vectors_b_noisy = [vsr.Vec(*(np.array(vector)
                            + sigma * np.random.randn(3))).null() 
                  for vector in vectors_b]

In [ ]:
vectors_a = [vsr.Vec(1,0,0), vsr.Vec(0,1,0), vsr.Vec(0,0,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [ ]:
ans = [hyperdual.hyperdualB(a,b,vsr.Biv(0,pi/6,0).exp()) for a,b in zip(vectors_a, vectors_b)]
res = np.sum([t[0] for t in ans])
grad = np.sum([t[1] for t in ans], axis=0)[1:]
Hess = np.sum([t[2] for t in ans], axis=0)

In [ ]:
grad

In [ ]:
ans

In [ ]:
print(Hess)

In [ ]:
aegas = [vsr.EGA(a) for a in vectors_a]
begas = [vsr.EGA(b) for b in vectors_b]
aegas

In [ ]:
begas

In [ ]:
a = aegas[0]
b = begas[0]

In [ ]:
R = vsr.EGA(vsr.Biv(0,pi/6,0).exp())
R

In [ ]:
B = vsr.EGA(vsr.Biv(1,0,0))
gr = vsr.EGA(vsr.Biv(*ans[0][1][1:]))
gr
rar = a.spin(R)

In [ ]:
B * a.spin(R) * b - a.spin(R) * B * b

In [ ]:
B * a.spin(R) * b - b * a.spin(R) * B

In [ ]:
B * a.spin(R) * b + B * a.spin(R) * b

In [ ]:
(B * 2) * a.spin(R) * b

In [ ]:
2 * a.spin(R) ^ b

In [ ]:
a.spin(R) * b - b * a.spin(R)

In [ ]:
gr <= B

In [ ]:
B = vsr.EGA(vsr.Biv(1,0,0))
2 * rar * B.rev() * b

In [ ]:
vsr.EGA(a * b) - vsr.EGA(b * a)

In [ ]:
(2 * rar * B.rev() * b) <= vsr.EGA(vsr.Biv(0,1,0))

In [ ]:
def Hij(Bi,Bj,a,b,rot): 
    ar = a.spin(rot)
    ans = (Bi * Bj * ar * b + 
#            Bi * ar * Bj.rev() * b + 
#            Bi* ar * Bj.rev() * b +
           ar * Bi.rev() * Bj.rev() * b) 
    return -ans[0] 

In [ ]:
def Hij2(Bi,Bj,a,b,rot): 
    ar = a.spin(rot)
    ans = (
           Bi * ar * Bj.rev() * b + 
           Bi* ar * Bj.rev() * b
        )
    return -ans[0] 

In [ ]:
def Hess2(a,b,rot):
    a = vsr.EGA(a)
    b = vsr.EGA(b)
    R = vsr.EGA(rot)
    E12 = vsr.EGA(vsr.Biv(1,0,0))
    E13 = vsr.EGA(vsr.Biv(0,1,0))
    E23 = vsr.EGA(vsr.Biv(0,0,1))
    return np.array([[Hij(E12,E12,a,b,R), Hij(E12,E13,a,b,R),Hij(E12,E23,a,b,R)],
                     [Hij(E13,E12,a,b,R), Hij(E13,E13,a,b,R),Hij(E13,E23,a,b,R)],
                     [Hij(E23,E12,a,b,R), Hij(E23,E13,a,b,R),Hij(E23,E23,a,b,R)]])

In [ ]:
def Hess3(a,b,rot):
    a = vsr.EGA(a)
    b = vsr.EGA(b)
    R = vsr.EGA(rot)
    E12 = vsr.EGA(vsr.Biv(1,0,0))
    E13 = vsr.EGA(vsr.Biv(0,1,0))
    E23 = vsr.EGA(vsr.Biv(0,0,1))
    return np.array([[Hij2(E12,E12,a,b,R), Hij2(E12,E13,a,b,R),Hij2(E12,E23,a,b,R)],
                     [Hij2(E13,E12,a,b,R), Hij2(E13,E13,a,b,R),Hij2(E13,E23,a,b,R)],
                     [Hij2(E23,E12,a,b,R), Hij2(E23,E13,a,b,R),Hij2(E23,E23,a,b,R)]])

In [ ]:
def Hess4(a,b,rot):
    return Hess2(a,b,rot) - Hess3(a,b,rot)

In [ ]:
def grad2(a,b,rot):
    def grad_ij(B, a ,b,rot):
        return (B.comm(a.spin(rot)) * b * -2)[0]
    return vsr.Biv(grad_ij(vsr.Biv(1,0,0),a,b,rot),
                   grad_ij(vsr.Biv(0,1,0),a,b,rot),
                   grad_ij(vsr.Biv(0,0,1),a,b,rot))

In [ ]:
a = vsr.Vec(0,1,0)
b = a.spin((vsr.Biv(1,0,0).unit() * (pi/6)).exp())
R = (vsr.Biv(0,1,0).unit() * (pi/6)).exp()

In [ ]:
a = vsr.EGA(a)
b = vsr.EGA(b)
R = vsr.EGA(R)
ar = a.spin(R)
E12 = vsr.EGA(vsr.Biv(1,0,0))
E13 = vsr.EGA(vsr.Biv(0,1,0))
E23 = vsr.EGA(vsr.Biv(0,0,1))

In [ ]:
B1 = E12
B2 = E12

In [ ]:
(B1 * B2 * ar * b + 
 B1 * ar * B2.rev() * b + 
 B2* ar * B1.rev() * b +
 ar * B1.rev() * B2.rev() * b) * 0.5

In [ ]:
B1 * (B2 * ar * b - b * ar * B2.rev() * 2.0 + ar.spin(B) * b * 2.0)

In [ ]:
a = vsr.Vec(1,0,0)
b = a.spin((vsr.Biv(1,0,0).unit() * (pi/6)).exp())
R = (vsr.Biv(0,1,1).unit() * (pi/6)).exp()

In [ ]:
a = vsr.Vec(1,1,0)
print(a.norm())
b = a.spin((vsr.Biv(1,0,0).unit() * (pi/6)).exp())

# print((a * a)[0])
R = (vsr.Biv(0,1,0).unit() * (pi/6)).exp()

# R = rotor
print(Hess2(a,b,R),'\n')
print(Hess3(a,b,R),'\n')
print(Hess4(a,b,R),'\n')
print(hyperdual.hyperdualB(a,b,R)[2])

In [ ]:
(hyperdual.hyperdualB(a,b,R)[2] + Hess4(a,b,R) ) / 4

In [ ]:
a.norm() 

In [ ]:
H = ans[0][2]
np.dot(np.dot(np.array([1,0,0]).reshape(3,1).T,H),np.array([0,1,0]).reshape(3,1))

In [ ]:
cos(pi/6) - (pi / 9 )

In [ ]:
# R = vsr.Biv(0,pi/4,0).exp()
(Hess2(a,b,R).copy()-hyperdual.hyperdualB(a,b,R)[2].copy() )

In [ ]:
a.spin(R) <= b

In [ ]:
2 * pi /4

In [ ]:
cos(pi / 1)

In [ ]:
print(Hess2(a,b,R).copy()[0,0])
print(hyperdual.hyperdualB(a,b,R)[2].copy()[0,0])

In [ ]:
ans[0][2]

In [ ]:
((vsr.Biv(1,2,3).unit() * pi/6).exp() * rotor).rev()

In [ ]:
rotor.rev() * (vsr.Biv(1,2,3).unit() * pi/6).exp().rev()

In [ ]:
E = np.array([1,0,0]).reshape(3,1)
np.dot(np.dot(E.T, H2), E)

In [ ]:
rotor = (vsr.Biv(1,2,3).unit() * pi/6).exp()
a = vsr.Vec(1,0,0)
b = a.spin(rotor)
rotor1 = vsr.Biv(0,0,pi/6).exp()
_,_,H = hyperdual.hyperdualB(a,b,rotor1)
print(H)

In [ ]:
Hess2(vsr.EGA(a), vsr.EGA(b), vsr.EGA(rotor1))

In [ ]:
E12 * cos(pi/6)

In [ ]:
Hij()

In [ ]:
vectors_a = [vsr.Vec(1,0,0), vsr.Vec(0,1,0), vsr.Vec(0,0,1)]
vectors_b = [a.spin(rotor) for a in vectors_a]

In [ ]:
n_points=10
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]

In [ ]:
rot = vsr.Rot(1,0,0,0)
np.sum([np.array(Hess2(a,b,rot)) for a,b in zip(vectors_a, vectors_b)],axis=0)

In [ ]:
ans = [hyperdual.hyperdualB(a,b,rot) for a,b in zip(vectors_a, vectors_b)]
res = np.sum([t[0] for t in ans])
grad = np.sum([t[1] for t in ans], axis=0)[1:]
Hess = np.sum([t[2] for t in ans], axis=0)
Hess

In [ ]:
def update(rot):
    ans = [hyperdual.hyperdualB(a,b,rot) for a,b in zip(vectors_a, vectors_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)[1:]
    Hess = np.sum([t[2] for t in ans], axis=0)
    
#     print(Hess)
    Hess = -np.sum([np.array(Hess2(a,b,rot)) for a,b in zip(vectors_a, vectors_b)],axis=0)
#     print(Hess)
    B = np.dot(np.linalg.pinv(Hess),grad)
    rot = vsr.Biv(*B).exp() * rot
#     rot = vsr.Biv(*(-grad * 0.1)).exp() * rot
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [ ]:
rot = vsr.Rot(1,0,0,0)
for i in range(10):
    rot, res, grad_norm = update(rot)
    print(res)
print(rot)

In [ ]:
rot = vsr.Rot(1,0,0,0)
J = np.array([jacrot(a,rot) for a in vectors_a]).reshape(-1,3)
print(np.dot(J.T,J))

In [ ]:
Hess = np.sum([hyperdual.hyperdualB(a,b,rot)[2] for a,b in zip(vectors_a, vectors_b)], axis=0)
print(Hess)

In [ ]:
motor1 = vsr.Mot(1,0,0,0,0,0,0,0)
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
points_a = [a.null() for a in vectors_a]
points_b = [a.spin(motor) for a in points_a]
a = points_a[0]
b = points_b[0]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

In [ ]:
motor0 = vsr.Mot(1,0,0,0,0,0,0,0)
r,g,H = hyperdual.hyperdualmotor(a,b, motor0)
# print(g)
print(H)

In [ ]:
ans = [hyperdual.hyperdualmotor(a,b,motor0) for a,b in zip(points_a, points_b_noisy)]
res = np.sum([t[0] for t in ans])
grad = np.sum([t[1] for t in ans], axis=0)
Hess = np.sum([t[2] for t in ans], axis=0)

In [ ]:
print(grad * -2) 

In [ ]:
vsr.Dll(*grad).spin(motor0)

In [ ]:
print(np.array(motor0 * vsr.Dll(*(np.sum([np.array(vsr.CGA(a.spin(motor0) ^ b) * 2)[6:12]
                        for a, b in zip(points_a, points_b_noisy)], axis=0)) * 2)))

In [ ]:
np.array(vsr.CGA(points_a[0].spin(motor0) ^ points_b[0] ) * 2)[6:16]

In [ ]:
grad * 2

In [ ]:
print(Hess)

In [ ]:
motor.dll()

In [ ]:
scene = Scene(children=
              [point_mesh(point,color=Colors.HOTPINK) for point in points_a] +\
              [point_mesh(point,color=Colors.LIGHTSKYBLUE) for point in points_b] +\
              [line_mesh(vsr.Dll(*grad ))] +\
              [line_mesh(motor.dll())] +\
#               [point_mesh(point,color='palegreen') for point in points_b_noisy] +\
#               [point_mesh(point.spin(motors[1]),color='palegreen') for point in points_a] +\
#               [point_mesh(point.spin(em_point_3_res_noise),color=Colors.HOTPINK) for point in points_a] +\
#             [point_mesh(point.spin(vd_em_points_noise),color=Colors.BLACK) for point in points_a] +\

              [frame_mesh(),AmbientLight(color='#cccccc')])

camera = PerspectiveCamera(position=[3, -3, 3])
renderer = Renderer(camera=camera, 
                    scene=scene, 
                    controls=[TrackballControls(controlling=camera)],
                    background = 'white',
                    width='960',
                    height='512')
# renderer.width = '1024'
# renderer.height = '768'
display(renderer)

In [ ]:
def update(rot):
    ans = [hyperdual.hyperdualmotor(a,b,rot) for a,b in zip(points_a, points_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)
    Hess = np.sum([t[2] for t in ans], axis=0)
    B = -np.dot(np.linalg.pinv(Hess),grad)
    rot = vsr.Dll(*B).exp() * rot
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [ ]:
M = motor + vsr.Mot(0,2,0,0,0,0,0,0)
M * M.rev()

In [ ]:
M2 = M.retract()
M2 * M2.rev()

In [ ]:
def update2(rot):
    ans = [hyperdual.hyperdualmotor(a,b,rot) for a,b in zip(points_a, points_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)
    Hess = np.sum([t[2] for t in ans], axis=0)
    B = np.dot(np.linalg.pinv(Hess),grad)
    B = vsr.Mot(0,B[0], B[1], B[2], B[3], B[4], B[5], 0)
    rot = (rot * B ).retract()
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [ ]:
rot = vsr.Mot(1,0,0,0,0,0,0,0)
res = 1e6
i = 0
# while res > 1e-6:
for i in range(10):
    i += 1
    rot, res, grad_norm = update(rot)
    print(res)
    print(rot)
# print(i)

In [ ]:
np.linalg.norm(rot.rev() * motor)

In [ ]:
print(rot.rev() * motor)

In [ ]:
import motor_jacobian

In [ ]:
def jacmot(a,mot):
    j = np.zeros((3,6))
    for i in range(6):
        dll = vsr.Dll(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        dll[i] = 1.0
        j[:,i] = 2. * np.array(dll.comm(a.spin(mot)))[:3]
    return j

In [ ]:
J = np.array([jacmot(a,motor0) for a in points_a]).reshape(-1,6)

In [ ]:
np.dot(J.T,J)

In [ ]:
n_points = 10
sigma = 0.09
points_a = [(vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null(),
             vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null()  )
            for i in range(n_points)]

points_a_noisy = [(vsr.Vec(*(np.array(a)[:3] + sigma * np.random.randn(3))).null(),
                   vsr.Vec(*(np.array(b)[:3] + sigma * np.random.randn(3))).null()) 
                  for a,b in points_a]

lines_a = [vsr.Dll(a,b).unit() for (a,b) in points_a]
lines_b = [line.spin(motor) for line in lines_a]

In [ ]:
def jac(a, mot):
    j = np.zeros((6,6))
    for i in range(6):
        ei = vsr.Dll(0.0, 0.0, 0.0,0.0,0.0,0.0)
        ei[i] = 1.0
        j[:,i] = 2.0 * np.array(ei.comm(a.spin(mot))) # Wrong sign?
    return j

In [ ]:
J = np.array([jac(a, motor0) for a in lines_a]).reshape(-1,6)

In [ ]:
np.dot(J.T,J)

In [ ]:
f = np.array([np.array(a.spin(motor0) - b) for a,b in zip(lines_a, lines_b)]).reshape(-1,1)

In [ ]:
vsr.Dll(*np.dot(J.T,f))

In [ ]:
motor.dll()

In [ ]:
np.sum([np.array(a.spin(motor0).comm(b)) for a,b in zip(lines_a, lines_b)],axis=0)

In [ ]:
lines_a[0].spin(motor).rev()

In [ ]:
lines_a[0].rev().spin(motor)

In [ ]:
l = lines_a[0].spin(motor0) - lines_b[0]
np.inner(l,l)

In [ ]:
np.sum([a*a for a in np.array(l)])

In [ ]:
l * l.rev()

In [ ]:
(l.biv() * l.biv().rev() )[0] + (l.drv().vec() * l.drv().vec().rev())[0]

In [ ]:
Ba = lines_a[0]
Ba[3] = Ba[4] = Ba[5] = 0 
Ba

In [ ]:
da = lines_a[0]
da[0] = da[1] = da[2] = 0
da

In [ ]:
Ba.spin(motor)

In [ ]:
from hyperdual_lines import lines

In [ ]:
lc = np.array(lines_a[0].spin(motor0) - lines_b[0])

In [ ]:
lc[0] * lc[0]

In [ ]:
lines(lines_a[0], lines_b[0], motor0)

In [ ]:
ans = [lines(a,b,motor0) for a,b in zip(lines_a, lines_b)]
res = np.sum([t[0] for t in ans])
grad = np.sum([t[1] for t in ans], axis=0)
Hess = np.sum([t[2] for t in ans], axis=0)
print(grad)

In [ ]:
vsr.Dll(*grad).spin(motor0)

In [ ]:
(motor0 * vsr.Dll(*np.sum([np.array(a.spin(motor0).comm(b)) 
                          for a,b in zip(lines_a, lines_b)],axis=0))).dll()

In [ ]:
vsr.Dll(*np.sum([np.array(a.spin(motor0).comm(b)) 
                          for a,b in zip(lines_a, lines_b)],axis=0))

In [ ]:
n_points = 10
sigma = 0.09
points_a = [(vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null(),
             vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null()  )
            for i in range(n_points)]

points_a_noisy = [(vsr.Vec(*(np.array(a)[:3] + sigma * np.random.randn(3))).null(),
                   vsr.Vec(*(np.array(b)[:3] + sigma * np.random.randn(3))).null()) 
                  for a,b in points_a]

lines_a = [vsr.Dll(a,b).unit() for (a,b) in points_a]
lines_b = [line.spin(motor) for line in lines_a]

In [ ]:
def update(rot):
    ans = [lines(a,b,rot) for a,b in zip(lines_a, lines_b)]
    res = np.sum([t[0] for t in ans])
    grad = np.sum([t[1] for t in ans], axis=0)
    Hess = np.sum([t[2] for t in ans], axis=0)
    B = -np.dot(np.linalg.pinv(Hess),grad)
    rot = vsr.Dll(*B).exp() * rot
    grad_norm = np.linalg.norm(grad)
    return rot, res, grad_norm

In [ ]:
rot = vsr.Mot(1,0,0,0,0,0,0,0)
res = 1e6
i = 0
ress = []
while res > 1e-3:
# for i in range(100):
    i += 1
    rot, res, grad_norm = update(rot)
    ress.append(res)
print(ress)
print(rot)
print(i)

In [ ]:
plot(ress);